In [5]:
import numpy as np 


In [6]:

def binomial_tree_option(N,T,S0,sigma,r,comp, K, call = True, array_out = False, compounding = True):
    """ Description

    Binomial tree option pricing 

    Parameters 

    N = periods, T = time to maturity, S0 = spot price, sigma = volatility, r = interest rate, K = strike price, call = True(else put)
    array_out = False just the price, else binomial tree, prices of the option in each period and the price as the output
    compounding = True(continuous), else(not continuous)

    Returns 

    Call OR put option price
    """
    dt = T/N
    u = np.exp(sigma*np.sqrt(dt))
    d = 1/u
    if compounding:
        p = (np.exp(r*dt)-d)/(u-d)
    else:
        p = (((1+r/comp)**dt*comp)-d)/(u-d)
    #Binomial Tree
    price_tree = np.zeros([N+1, N+1])
    for i in range(N+1):
        for j in range(i+1):
            price_tree[j,i] = S0*(d**j)*(u**(i-j))
    #Option value 
    option = np.zeros([N+1, N+1])
    if call:
        option[:,N] = np.maximum(np.zeros(N+1), price_tree[:,N]-K)
    else:
        option[:,N] = np.maximum(np.zeros(N+1), K - price_tree[:,N])
    #Calculate option price at t = 0

    for i in np.arange(N-1, -1,-1):
        for j in np.arange(0,i+1):
            if compounding: 
                option[j,i] = np.exp(-r*dt)*(p*option[j,i+1]+(1-p)*option[j+1, i+1])
            else: 
                option[j,i] = (p*option[j,i+1]+(1-p)*option[j+1, i+1])/((1+r/comp)**dt*comp)
    # Return
    if array_out:
        return[option[0,0], price_tree, option]
    else: 
        return option[0,0]


In [7]:
binomial_tree_option(N=4,T = 1,S0 = 100,sigma = 0.1,r = 0.03,comp = 1, K = 110, call = True, array_out = False, compounding = False)

1.3287736652343096

In [8]:

def binomial_tree_call_put(N,T,S0,u, d, r,comp, K, call = True, array_out = False, compounding = True):
    """ 
    Description:

    Binomial tree option pricing 

    Parameters:

    N = periods, T = time to maturity, S0 = spot price, u = up movement as percentage of the t-1 period, d = down movement as         percentage of t-1 period, r = interest rate, K = strike price, call = True(else put)

    array_out = False just the price, else binomial tree, prices of the option in each period and the price as the output
    
    compounding = True(continuous), else(not continuous)

    Returns:
    Call OR put option price
    """
    dt = T/N
    print(dt)
    if compounding:
        p = (np.exp(r*dt)-d)/(u-d)
    else:
        p = (((1+r_fc/comp)**dt*comp)-d)/(u-d)
    print(p)
    #Binomial Tree
    price_tree = np.zeros([N+1, N+1])
    for i in range(N+1):
        for j in range(i+1):
            price_tree[j,i] = S0*(d**j)*(u**(i-j))
    #Option value 
    option = np.zeros([N+1, N+1])
    if call:
        option[:,N] = np.maximum(np.zeros(N+1), price_tree[:,N]-K)
    else:
        option[:,N] = np.maximum(np.zeros(N+1), K - price_tree[:,N])
    #Calculate option price at t = 0
    for i in np.arange(N-1, -1,-1):
        for j in np.arange(0,i+1):
            if compounding: 
                option[j,i] = np.exp(-r*dt)*(p*option[j,i+1]+(1-p)*option[j+1, i+1])
            else: 
                option[j,i] = (p*option[j,i+1]+(1-p)*option[j+1, i+1])/((1+r/comp)**dt*comp)
    # Return
    if array_out:
        return[option[0,0], price_tree, option]
    else: 
        return option[0,0]


In [9]:
binomial_tree_call_put(N=1,T = 2,S0 = 100,u = 1.3, d = 0.8,r = 0.1, comp = 1, K = 103, call = True, array_out = True, compounding = False)

2.0


NameError: name 'r_fc' is not defined

In [18]:

def binomial_tree_call_put_currency(N,T,S0,u, d, r, r_fc, comp, K, call = True, array_out = False, compounding = True):
    """ 
    Description:

    Binomial tree option pricing 

    Parameters:

    N = periods, T = time to maturity, S0 = spot price, u = up movement as percentage of the t-1 period, d = down movement as         percentage of t-1 period, r = interest rate, K = strike price, call = True(else put)

    array_out = False just the price, else binomial tree, prices of the option in each period and the price as the output
    
    compounding = True(continuous), else(not continuous)

    Returns:
    Call OR put option price
    """
    dt = T/N
    print(dt)
    if compounding:
        p = (np.exp(r*dt)-d)/(u-d)
    else:
        p = (((1+r_fc/comp)**(dt*comp))-d)/(u-d)
    print(p)
    #Binomial Tree
    price_tree = np.zeros([N+1, N+1])
    for i in range(N+1):
        for j in range(i+1):
            price_tree[j,i] = S0*(d**j)*(u**(i-j))
    #Option value 
    option = np.zeros([N+1, N+1])
    if call:
        option[:,N] = np.maximum(np.zeros(N+1), price_tree[:,N]-K)
    else:
        option[:,N] = np.maximum(np.zeros(N+1), K - price_tree[:,N])
    #Calculate option price at t = 0
    for i in np.arange(N-1, -1,-1):
        for j in np.arange(0,i+1):
            if compounding: 
                option[j,i] = np.exp(-r*dt)*(p*option[j,i+1]+(1-p)*option[j+1, i+1])
            else: 
                option[j,i] = (p*option[j,i+1]+(1-p)*option[j+1, i+1])/((1+r/comp)**dt*comp)
    # Return
    if array_out:
        return[option[0,0], price_tree, option]
    else: 

        return option[0,0]
        return option[0,0]


In [23]:
binomial_tree_call_put_currency(N=1,T = 2,S0 = 100,u = 1.3, d = 0.8,r = 0.1, r_fc = 0.1, comp = 1, K = 100, call = False, array_out = True, compounding = False)

2.0
0.8200000000000003


[2.975206611570243,
 array([[100., 130.],
        [  0.,  80.]]),
 array([[ 2.97520661,  0.        ],
        [ 0.        , 20.        ]])]